# Tutoriel sql avec postgresql

*Adaptation du tutoriel officiel de postgresql*

## Avant toute chose

**Commencer par ouvrir un terminal et placer le à côté de ce document, puis**:

1. créer un répertoire *sql* contenant un répertoire *tuto* dans votre espace,
   
```
~$ mkdir -p sql/tuto # -p pour path (chemin): crée tous les dossiers du chemin
~$ cd sql/tuto
```

2. puis y copier les deux fichiers du répertoire *1_decouverte_sql* qui contient ce notebook (`*` signifie «tous» et `.` dénote le répertoire courant),

```
~/sql/tuto$ cp ~/.../1_decouverte/* .
```

3. **créer la base de donnée**: préfixer le nom de la base par vos initiales `<initiales>_test_bd`,

```
~/sql/tuto$ createdb <initiales>_test_bd
```

4. se connecter à cette base avec le client *psql*.

```
~/sql/tuto$ psql <initiales>_test_bd
```

Vous obtenez alors un **prompt**:

    test_bd=> 

à la suite duquel on peut saisir des **commandes sql** ou des «pseudos commandes» expliquées ci-après.

## `CREATE TABLE` - Créer une nouvelle table

On utilise une instruction de la *forme*:   

Noter le `;` qui indique la fin de l'instruction sql: il est obligatoire!

Noter encore que SQL n'est pas sensible à la «casse»: `CREATE` ou `Create` ou `create` se réfèrent à la même commande; préférer les minuscules car c'est plus facile à saisir.

Enfin, les espaces et sauts de lignes équivalent à un espace seul: cela permet de *formater agréablement* les «ordres sql».

Par exemple (`--` annonce un commentaire - inutile de le saisir!):

```sql
CREATE TABLE meteos (
    ville    varchar(80),
    t_min    int,     -- température minimale
    t_max    int,     -- température maximale
    prcp     real,    -- précipitation
    date     date
);
```

Cela crée une table `meteo` ayant 5 colonnes. 

Vérifier-le avec les «pseudos commandes» `\d` (pour *describe*) puis `\d meteos` (saisir `q` pour quitter).

#### À vous de jouer

Créer une table **villes** dont psql donne la description suivante (avec `\d villes`):

*Note*: `character varying(80)` et `varchar(80)` sont deux synonymes.

Vérifier vos tables avec `\d` et `\d villes`. 

**Solution**

```sql
CREATE TABLE villes (
    nom      varchar(80),
    position point -- type particulier à postgresql
);
```

*Note*: Si l'une de vos tables n'est pas correcte, supprimer la et recréer la en reprenant soigneusement le code sql fourni plus tôt:

```sql
DROP TABLE nom_table; -- supprime la table indiquée
```

## `INSERT INTO` - Ajouter des lignes

L'instruction est de la *forme*:   

*Note*: Les crochets indiquent un ou des arguments *optionnels*.

Ainsi, il y a plusieurs façons de préciser à quelle colonne correspond quelle valeur.

**Première façon**: Le cas le plus simple consiste à donner les valeurs dans le même ordre que celui des colonnes dans la table (lors de sa création). Par exemple:

```sql
INSERT INTO meteos VALUES ('Tours', 12, 23, 0.2, '2020-11-03');
```

Note: l'instruction `select * from meteos;` permet de vérifier l'insertion. 

#### À vous de jouer

La ville de `'Tours'` a pour coordonnées (long, lat) `'(47.38333, 0.68333)'`. Insérer une ligne dans la table **villes** pour tours.

**Solution**

```sql
INSERT INTO villes VALUES ('Tours', '(47.38333, 0.68333)');
-- vérification
select * from villes;
```

**Deuxième façon**: On peut préciser les colonnes associées aux valeurs après le nom de la table; les valeurs doivent alors être données dans le même ordre. Par exemple:

```sql
INSERT INTO meteos (t_min, t_max, date, ville, prcp) 
    VALUES        (10, 18, '2020-11-04', 'Tours', 0.2);
```

#### À vous de jouer

> La journée du 15 novembre 2020, à bordeaux, le temps était sec et la température a varié de 12 à 19 degrés.

Insérer cette information dans la table appropriée.

**Solution**

```sql
INSERT INTO meteos (date, ville, prc, t_min, t_max)
VALUES            ('2020-11-15', 'Bordeaux', 0, 12, 19);
-- vérification
select * from meteos;
```

## Insertion de données «en masse»

Pour pouvoir découvrir la notion de **requête dans une base de données**, nous allons insérer des données via un **script sql**.

**Dans le hub**, ouvrir le fichier *tuto.sql* pour voir son contenu,

**Dans psql** (terminal), utiliser la pseudo commande `\i` pour «jouer» le script sql:

    test_bd=>\i tuto.sql

*Note*: le script *tuto.sql* doit être dans le répertoire courant de *psql*. Si ce n'est pas le cas, quitter psql avec `\q` et le rouvrir depuis le répertoire approprié.

## `SELECT` - Requête sur une table

L'instruction `SELECT` est utilisée pour récupérer/explorer des données. 
Sa syntaxe (simplifié) est de la forme:

- Voici un exemple simple (`*` signifie «toutes les colonnes de la table»):

```sql
SELECT * FROM meteos;
```

Vous devriez obtenir l'affichage:

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Afficher toutes les lignes de la table villes.

```sql
SELECT * FROM villes;
```

- On peut aussi mettre des expressions à la place d'une colonne:

```sql
SELECT ville, (t_min+t_max)/2 AS t_moy, date 
FROM meteos;
```

Note: Le mot clé `AS` sert à créer un «alias» de colonne pour l'expression.

Vous devriez obtenir l'affichage:

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Afficher l'écart de température pour chaque ligne de la table. Ne conserver que les colonnes ville, date et écart.

```sql
SELECT ville, date, (t_max-t_min) AS écart
FROM meteos;
```

- Si vous souhaitez sélectionner les lignes qui satisfont à certaines **conditions**, placer les après `WHERE`. Par exemple:

```sql
SELECT * FROM meteos
WHERE ville = 'Paris'
  AND prcp > 0.0;
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Afficher les températures min pour la ville de 'Tours' à partir du 7 novembre 2020.

```sql
SELECT t_min
FROM meteos
WHERE ville = 'Tours'
  AND date >= '2020-11-07';
```

- Tester *successivement* (en les tapant - ok pour l'historique...) les instructions SQL suivantes:

```sql
SELECT ville 
FROM meteos;

SELECT DISTINCT ville
FROM meteos;

SELECT ville
FROM meteos
ORDER BY ville;
```

### Jointure entre tables

- Une **requête** peut porter sur plusieurs table à la fois. Par exemple:

```sql
SELECT *
FROM meteos, villes;
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Combien obtient-on de lignes? Pourquoi? (aide: regarder le nombre de lignes de chaque tables)

Normalement **72** ($= 18\times 4$). En effet chaque ligne de la table meteos (il y en a 18) est combinée avec chaque ligne de la table villes (il y en a 4).

- Pour éviter ce phénomène on cherche *une colonne de chaque table* dont les valeurs sont «comparables» (notamment de même type) afin de **joindre** les tables et de «croiser» leurs données:

```sql
SELECT *
FROM meteos, villes
WHERE ville = nom;
```

Éliminons la duplication de 'Tours' et raccourcissons:

```sql
SELECT ville, t_min, t_max, position
FROM meteos, villes
WHERE ville = nom;
```

Comme les noms de colonnes sont *tous différents d'une table à l'autre*, on n'a pas besoin de préciser à quelle table appartient une colonne ici. En pratique c'est rarement le cas et il faudrait lever l'ambiguîté en écrivant plutôt quelque chose comme:

```sql
SELECT M.ville, M.t_min, M.t_max, V.position
FROM meteos M, villes V -- alias de tables
WHERE M.ville = V.nom;
```

cela permet de préciser l'appartenance d'une colonne à une table en utilisant un **nom qualifié**.

- On peut encore **joindre une table à elle-même**.

Dans ce cas il est *indispensable* d'utiliser des noms qualifiés.

Supposons qu'on veuille trouver toutes les lignes de meteos dont la température min est supérieure à la température max d'au moins une autre ligne (ou elle-même).

```sql
SELECT M1.ville, M1.t_min, M1.date, M2.ville, M2.t_max, M2.date
FROM meteos M1, meteos M2
WHERE M1.t_min > M2.t_max
  AND M1.date = M2.date;
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span>Lors de la saisie des données, un opérateur a pu se tromper. Par exemple, une ligne pourrait contenir une température min inférieure à la température max.

Chercher si une telle ligne existe en joignant la table meteos à elle-même.

```sql
SELECT M1.* FROM meteos M1, meteos M2
WHERE M1.t_min > M2.t_max 
  AND M1.ville = M2.ville 
  AND M1.date = M2.date;
```

### Fonctions d'agrégation

L'agrégation consiste à **combiner les valeurs d'une colonne** d'une table afin d'obtenir un résumé. Voici un exemple avec la fonction `max`:

```sql
SELECT max(t_min)
FROM meteos;
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Voici quelques fonctions d'aggrégation à tester: `min`, `avg` (*average*, moyenne), `sum`, `count` (souvent utilisé sous la forme `count(*)`), ...
_____

Une agrégation produit normalement une seule valeur et on peut donc l'utiliser comme **sous-requête** pour compléter une condition:

```sql
SELECT ville, t_min, date FROM meteos
WHERE t_min = (SELECT max(t_min) FROM meteos);
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Trouver la ville qui a le «record de température» et le jour où cela a eu lieu.

```sql
SELECT ville, date
FROM meteos
WHERE t_max = (
    select max(t_max) from meteos
);
```

Plutôt que d'aggréger toutes les lignes d'une table, on peut aggréger des **groupes de lignes**:

```sql
SELECT ville, max(t_min)
FROM meteos
GROUP BY ville;
```

Il est possible de sélectionner/filtrer certains de ces groupes en utilisant l'**agrégat** avec `HAVING`:

```sql
SELECT ville, max(t_min)
FROM meteos
GROUP BY ville
HAVING max(t_min) < 16;
```

## `UPDATE` - Mettre à jour une table

Supposer que nous découvrions que toutes les températures ait été surestimées de 2 degrés à partir du 4 novembre 2020: 

```sql
UPDATE meteos
SET t_max = t_max - 2,  t_min = t_min - 2
WHERE date >= '2020-11-04';
```

#### À vous de jouer

<span style="font-size: 18pt;">☞</span> Comme vue prédémment, une ligne est erroné (t_min > t_max). On suppose que les températures ont été inversées.

Corriger cette erreur.

## ̀`DELETE` - Suppression de lignes

Supposez que nous ne soyons plus intéressés par les données météos de Paris:

```sql
DELETE FROM meteos
WHERE ville = 'Paris';
```

On peut supprimer toutes les lignes d'une table (cela ne supprime pas la table, le vérifier):

```sql
DELETE FROM meteos;
```

## `DROP` - Suppression de tables

```sql
DROP TABLE meteos, villes;
```

## Quitter psql et supprimer la base de données

Saisir la commande `\q`

    test_bd=>\q

De retour dans le shell ordinaire:

    ...$ dropdb test_bd